In [1]:
import faiss
exec(open("../database.py").read())
exec(open("../helperFunctions.py").read())
import numpy as np
from keras.datasets import mnist
from time import perf_counter
from ast import literal_eval

In [2]:
datasetImages, datasetLabels = readMnist()

datasetImages :  (70000, 784)
datasetLabels :  (70000,)


***Create faiss***

In [3]:
def createIndex(indexMethod, datasetImages):
    d = datasetImages.shape[1] # dimension
    M = 16
    time_start = perf_counter()
    index = indexMethod(d, M)
    index.add(datasetImages) 
    time_end = perf_counter()
    totalTime = (time_end - time_start)
    print(f'Took {totalTime:.3f} seconds')
    return (index, totalTime)
(indexedStruct, time) = createIndex(faiss.IndexHNSWFlat, datasetImages)

Took 9.128 seconds


In [4]:
print(indexedStruct)

<faiss.swigfaiss.IndexHNSWFlat; proxy of <Swig Object of type 'faiss::IndexHNSWFlat *' at 0x000001BD776CED80> >


In [ ]:
# (min, max) = createIndexNumerous(createIndex, NearestNeighbors, datasetImages, 10)
# print('min : ', min, '\n','max : ', max,)

In [5]:
indexes = []
distances = []
def measureTime(par, indexes, distances):
    k=100
    totalTime = 0
    for i in range(par) : 
        xq = datasetImages[i:i+1].astype('float32') # Use the first image as the query vector
        time_start = perf_counter()
        distance, index = indexedStruct.search(xq, k) 
        time_end = perf_counter()
        totalTime += (time_end - time_start)
        distances.append(np.sqrt(distance[0]))
        indexes.append(index[0])
    # report the duration
    print(f'Took {totalTime:.3f} seconds')
measureTime(1000, indexes, distances)

Took 0.554 seconds


In [ ]:
# (min, max) = measureTimeNumerous(measureTime, 10)
# print('min : ', min, '\n', 'max : ', max,)

In [6]:
indexes = np.array(indexes)
distances = np.round(np.array(distances).astype(float), 4)

In [7]:
print('indexes : ', indexes.shape,'\n', 'distances : ', distances.shape,)

indexes :  (1000, 100) 
 distances :  (1000, 100)


In [8]:
(trueIndexes, trueDistances) = readDB()

trueIndexes :  (1000, 100)
trueDistances :  (1000, 100)


In [9]:
compareFirstTen(indexes, distances, trueIndexes, trueDistances)

0 || 0
0.0 || 0.0
8728 || 32248
1591.6014 || 1561.4724
18932 || 8728
1594.7191 || 1591.6014
63470 || 18932
1600.8738 || 1594.7191
65325 || 30483
1603.479 || 1596.7095
24149 || 63470
1604.4469 || 1600.8738
42338 || 65325
1604.8876 || 1603.479
52295 || 24149
1605.7092 || 1604.4469
26251 || 42338
1609.6943 || 1604.8876
50173 || 52295
1613.7047 || 1605.7092


In [10]:
calculateRecallAverage(indexes, distances, trueIndexes, trueDistances)
calculateRecallAverage(indexes, distances, trueIndexes, trueDistances, 1.01)
calculateRecallAverage(indexes, distances, trueIndexes, trueDistances, 1.1)

0.2160
0.4451
0.9823
